In [ ]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time


In [ ]:
# Load the overlay
overlay = Overlay('./radiation_overlay.bit')

# Access the radiation detector IP
radiation_detector = overlay.radiation_detector_core


In [ ]:
# Configure DMA buffer for data transfer
BUFFER_SIZE = 1024
input_buffer = allocate(shape=(BUFFER_SIZE,), dtype=np.uint32)
output_buffer = allocate(shape=(BUFFER_SIZE,), dtype=np.uint32)

# Configure detector parameters
def configure_detector(threshold=2048):
    radiation_detector.write(0x10, threshold)  # Set threshold
    radiation_detector.write(0x14, 1)         # Enable
    
# Read detector status
def read_status():
    status = radiation_detector.read(0x00)
    event_count = radiation_detector.read(0x04)
    return {
        'alert': bool(status & 0x1),
        'event_count': event_count
    }


In [ ]:
# Real-time monitoring function
def monitor_radiation(update_interval=1.0, max_time=60):
    start_time = time.time()
    times = []
    counts = []
    
    plt.figure(figsize=(12, 6))
    while time.time() - start_time < max_time:
        status = read_status()
        current_time = time.time() - start_time
        
        times.append(current_time)
        counts.append(status['event_count'])
        
        clear_output(wait=True)
        plt.plot(times, counts, 'b-')
        plt.xlabel('Time (s)')
        plt.ylabel('Event Count')
        plt.title('Radiation Events Over Time')
        plt.grid(True)
        plt.show()
        
        if status['alert']:
            print(f"⚠️ Alert! High radiation level detected at t={current_time:.1f}s")
            
        time.sleep(update_interval)
        
    return np.array(times), np.array(counts)
